## ChatNS

In [31]:
import heapq


def parse_maze(maze):
    start = None
    end = None
    grid = []
    for y, line in enumerate(maze.strip().split('\n')):
        row = list(line)
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)
        grid.append(row)
    return grid, start, end


def heuristic(pos, end):
    # Manhattan distance between the current position and the end.
    return abs(pos[0] - end[0]) + abs(pos[1] - end[1])


def get_neighbors(pos, direction, grid):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # East, North, West, South
    x, y = pos
    neighbors = []

    # Move forward
    dx, dy = directions[direction]
    nx, ny = x + dx, y + dy
    if 0 <= nx < len(grid[0]) and 0 <= ny < len(grid) and grid[ny][nx] != '#':
        neighbors.append(((nx, ny), direction, 1))  # Moving forward cost

    # Rotate clockwise
    neighbors.append(((x, y), (direction + 1) % 4, 1000))

    # Rotate counterclockwise
    neighbors.append(((x, y), (direction - 1) % 4, 1000))

    return neighbors


def find_lowest_score(maze):
    grid, start, end = parse_maze(maze)
    initial_direction = 0  # Start facing East
    start_state = (start, initial_direction, 0)  # (position, direction, cost)

    # Priority queue for A*
    priority_queue = []
    heapq.heappush(priority_queue, (0 + heuristic(start, end), 0, start_state))

    visited = {}

    while priority_queue:
        estimated_cost, current_cost, (current_pos, current_direction, _) = heapq.heappop(priority_queue)

        # If we have reached the end return the cost
        if current_pos == end:
            return current_cost

        # Check if we have visited this state with a better cost
        if (current_pos, current_direction) in visited and visited[(current_pos, current_direction)] <= current_cost:
            continue

        visited[(current_pos, current_direction)] = current_cost

        # Explore neighbors
        for neighbor_pos, new_direction, move_cost in get_neighbors(current_pos, current_direction, grid):
            new_cost = current_cost + move_cost
            new_estimate = new_cost + heuristic(neighbor_pos, end)
            new_state = (neighbor_pos, new_direction, new_cost)

            heapq.heappush(priority_queue, (new_estimate, new_cost, new_state))


# Example usage
maze_input = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""

second_example = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
"""

print(find_lowest_score(maze_input))
print(find_lowest_score(second_example))

7036
11048


In [32]:
with open('./data/Day 16/input.txt') as f:
    actual_input = f.read()

print(find_lowest_score(actual_input))

88416


In [33]:
import heapq
from collections import defaultdict


def parse_maze(maze):
    start = None
    end = None
    grid = []
    for y, line in enumerate(maze.strip().split("\n")):
        row = list(line)
        for x, cell in enumerate(row):
            if cell == "S":
                start = (x, y)
            elif cell == "E":
                end = (x, y)
        grid.append(row)
    return grid, start, end


def calculate_costs(grid, start):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # East, North, West, South
    rotate_cost = 1000

    pq = []
    heapq.heappush(pq, (0, start, 0))

    cost_map = defaultdict(lambda: float("inf"))
    cost_map[(start, 0)] = 0

    while pq:
        current_cost, (x, y), direction = heapq.heappop(pq)

        if cost_map[((x, y), direction)] < current_cost:
            continue

        dx, dy = directions[direction]
        nx, ny = x + dx, y + dy
        if 0 <= nx < len(grid[0]) and 0 <= ny < len(grid) and grid[ny][nx] != "#":
            new_cost = current_cost + 1
            if new_cost < cost_map[((nx, ny), direction)]:
                cost_map[((nx, ny), direction)] = new_cost
                heapq.heappush(pq, (new_cost, (nx, ny), direction))

        new_direction = (direction + 1) % 4
        if current_cost + rotate_cost < cost_map[((x, y), new_direction)]:
            cost_map[((x, y), new_direction)] = current_cost + rotate_cost
            heapq.heappush(pq, (current_cost + rotate_cost, (x, y), new_direction))

        new_direction = (direction - 1) % 4
        if current_cost + rotate_cost < cost_map[((x, y), new_direction)]:
            cost_map[((x, y), new_direction)] = current_cost + rotate_cost
            heapq.heappush(pq, (current_cost + rotate_cost, (x, y), new_direction))

    return cost_map


def trace_optimal_paths(grid, end, cost_map):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]
    rotate_cost = 1000
    valid_tiles = set()
    visited = set()

    min_cost = min(cost_map[(end, d)] for d in range(4))
    stack = [(end, d) for d in range(4) if cost_map[(end, d)] == min_cost]

    while stack:
        (x, y), direction = stack.pop()

        if ((x, y), direction) in visited:
            continue

        visited.add(((x, y), direction))
        valid_tiles.add((x, y))

        dx, dy = directions[direction]
        px, py = x - dx, y - dy

        if (
            0 <= px < len(grid[0])
            and 0 <= py < len(grid)
            and cost_map.get(((px, py), direction), float("inf")) + 1
            == cost_map[((x, y), direction)]
        ):
            stack.append(((px, py), direction))

        for rot, new_direction in [(1, (direction - 1) % 4), (-1, (direction + 1) % 4)]:
            if (
                cost_map.get(((x, y), new_direction), float("inf")) + rotate_cost
                == cost_map[((x, y), direction)]
            ):
                stack.append(((x, y), new_direction))

    return valid_tiles


def find_best_paths_tiles(maze_input):
    grid, start, end = parse_maze(maze_input)
    cost_map = calculate_costs(grid, start)
    valid_tiles = trace_optimal_paths(grid, end, cost_map)
    return len(valid_tiles)


# Test cases
maze_input1 = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""

maze_input2 = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
"""

print(
    f"Number of tiles part of any best path for maze 1: {find_best_paths_tiles(maze_input1)}"
)
print(
    f"Number of tiles part of any best path for maze 2: {find_best_paths_tiles(maze_input2)}"
)

Number of tiles part of any best path for maze 1: 45
Number of tiles part of any best path for maze 2: 64


In [34]:
find_best_paths_tiles(actual_input)

442

## System

You are an excellent python programmer with 10 years of experience. In this time you have excelled at solving problems with different algoritms, and are great at solving advent of code.

## User

It's time again for the Reindeer Olympics! This year, the big event is the Reindeer Maze, where the Reindeer compete for the lowest score.

You and The Historians arrive to search for the Chief right as the event is about to start. It wouldn't hurt to watch a little, right?

The Reindeer start on the Start Tile (marked S) facing East and need to reach the End Tile (marked E). They can move forward one tile at a time (increasing their score by 1 point), but never into a wall (#). They can also rotate clockwise or counterclockwise 90 degrees at a time (increasing their score by 1000 points).

To figure out the best place to sit, you start by grabbing a map (your puzzle input) from a nearby kiosk. For example:

###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############

There are many paths through this maze, but taking any of the best paths would incur a score of only 7036. This can be achieved by taking a total of 36 steps forward and turning 90 degrees a total of 7 times:


###############
#.......#....E#
#.#.###.#.###^#
#.....#.#...#^#
#.###.#####.#^#
#.#.#.......#^#
#.#.#####.###^#
#..>>>>>>>>v#^#
###^#.#####v#^#
#>>^#.....#v#^#
#^#.#.###.#v#^#
#^....#...#v#^#
#^###.#.#.#v#^#
#S..#.....#>>^#
###############

Here's a second example:

#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################

In this maze, the best paths cost 11048 points; following one such path would look like this:

#################
#...#...#...#..E#
#.#.#.#.#.#.#.#^#
#.#.#.#...#...#^#
#.#.#.#.###.#.#^#
#>>v#.#.#.....#^#
#^#v#.#.#.#####^#
#^#v..#.#.#>>>>^#
#^#v#####.#^###.#
#^#v#..>>>>^#...#
#^#v###^#####.###
#^#v#>>^#.....#.#
#^#v#^#####.###.#
#^#v#^........#.#
#^#v#^#########.#
#S#>>^..........#
#################

Note that the path shown above includes one 90 degree turn as the very first move, rotating the Reindeer from facing East to facing North.

Analyze your map carefully. What is the lowest score a Reindeer could possibly get?

## Assistant

To solve this problem, we'll need to implement a pathfinding algorithm that takes into account both the cost of moving forward and the higher cost of turning. A suitable algorithm for this kind of task is the A* algorithm, which is similar to Dijkstra's but uses heuristics to find the shortest path more efficiently.

Here's a step-by-step outline to solve this problem:

1. **Parse the Maze:** First, read the maze input and identify the start position ('S'), the end position ('E'), and any walls ('#') in the maze.

2. **Define Directions and Costs:** Define possible directions (East, North, West, South) and the respective turning costs. For moves:
   - Moving forward costs 1 point.
   - Turning right or left costs 1000 points.

3. **Heuristic for A* Algorithm:** Use Manhattan distance from the current position to the end position as a heuristic. This works well because it provides a good estimate of the minimum number of steps to reach the end if there were no obstacles.

4. **Implement A*:**
   - Use a priority queue to always expand the path with the lowest estimated total cost (current cost + heuristic).
   - Keep track of the score for each path, updating only if a lower score is found.

5. **State Representation:** At each step, keep track of the current position, the current facing direction, and the accumulated cost.

6. **Explore Moves:** For each state:
   - Attempt moving forward if possible and add resultant states to the priority queue.
   - Consider both clockwise and counterclockwise rotations, updating the direction and score appropriately.

7. **Store Result and Path:** When reaching the end ('E'), compare the score and retain the best (lowest score) path found.

Here's a simplified code implementation:

```python
import heapq


def parse_maze(maze):
    start = None
    end = None
    grid = []
    for y, line in enumerate(maze.strip().split('\n')):
        row = list(line)
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)
        grid.append(row)
    return grid, start, end


def heuristic(pos, end):
    # Manhattan distance between the current position and the end.
    return abs(pos[0] - end[0]) + abs(pos[1] - end[1])


def get_neighbors(pos, direction, grid):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # East, North, West, South
    x, y = pos
    neighbors = []

    # Move forward
    dx, dy = directions[direction]
    nx, ny = x + dx, y + dy
    if 0 <= nx < len(grid[0]) and 0 <= ny < len(grid) and grid[ny][nx] != '#':
        neighbors.append(((nx, ny), direction, 1))  # Moving forward cost

    # Rotate clockwise
    neighbors.append(((x, y), (direction + 1) % 4, 1000))

    # Rotate counterclockwise
    neighbors.append(((x, y), (direction - 1) % 4, 1000))

    return neighbors


def find_lowest_score(maze):
    grid, start, end = parse_maze(maze)
    initial_direction = 0  # Start facing East
    start_state = (start, initial_direction, 0)  # (position, direction, cost)

    # Priority queue for A*
    priority_queue = []
    heapq.heappush(priority_queue, (0 + heuristic(start, end), 0, start_state))

    visited = {}

    while priority_queue:
        estimated_cost, current_cost, (current_pos, current_direction, _) = heapq.heappop(priority_queue)

        # If we have reached the end return the cost
        if current_pos == end:
            return current_cost

        # Check if we have visited this state with a better cost
        if (current_pos, current_direction) in visited and visited[(current_pos, current_direction)] <= current_cost:
            continue

        visited[(current_pos, current_direction)] = current_cost

        # Explore neighbors
        for neighbor_pos, new_direction, move_cost in get_neighbors(current_pos, current_direction, grid):
            new_cost = current_cost + move_cost
            new_estimate = new_cost + heuristic(neighbor_pos, end)
            new_state = (neighbor_pos, new_direction, new_cost)

            heapq.heappush(priority_queue, (new_estimate, new_cost, new_state))


# Example usage
maze_input = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""

print(find_lowest_score(maze_input))
```

This code handles both moving forward and rotating, while prioritizing the lowest score path to reach the end. Adjust your algorithm and heuristic as necessary to suit other details that may arise in different problem configurations.

## User

Great job, now the challenge has extended:
Now that you know what the best paths look like, you can figure out the best spot to sit.

Every non-wall tile (S, ., or E) is equipped with places to sit along the edges of the tile. While determining which of these tiles would be the best spot to sit depends on a whole bunch of factors (how comfortable the seats are, how far away the bathrooms are, whether there's a pillar blocking your view, etc.), the most important factor is whether the tile is on one of the best paths through the maze. If you sit somewhere else, you'd miss all the action!

So, you'll need to determine which tiles are part of any best path through the maze, including the S and E tiles.

In the first example, there are 45 tiles (marked O) that are part of at least one of the various best paths through the maze:

###############
#.......#....O#
#.#.###.#.###O#
#.....#.#...#O#
#.###.#####.#O#
#.#.#.......#O#
#.#.#####.###O#
#..OOOOOOOOO#O#
###O#O#####O#O#
#OOO#O....#O#O#
#O#O#O###.#O#O#
#OOOOO#...#O#O#
#O###.#.#.#O#O#
#O..#.....#OOO#
###############

In the second example, there are 64 tiles that are part of at least one of the best paths:

#################
#...#...#...#..O#
#.#.#.#.#.#.#.#O#
#.#.#.#...#...#O#
#.#.#.#.###.#.#O#
#OOO#.#.#.....#O#
#O#O#.#.#.#####O#
#O#O..#.#.#OOOOO#
#O#O#####.#O###O#
#O#O#..OOOOO#OOO#
#O#O###O#####O###
#O#O#OOO#..OOO#.#
#O#O#O#####O###.#
#O#O#OOOOOOO..#.#
#O#O#O#########.#
#O#OOO..........#
#################

Analyze your map further. How many tiles are part of at least one of the best paths through the maze?

## Assistant

To determine which tiles are part of any best path through the maze, we'll need to keep track of all possible paths that result in the lowest score. This is essentially finding all paths from the start to the end that result in the lowest possible cost. Since A* can identify at least one optimal path, we'll modify it slightly to explore all equivalent optimal paths and then mark those tiles.

The key is to:

1. **Find all optimal paths**: Use a modified version of the A* algorithm that can track multiple paths with the same lowest cost.

2. **Track path tiles**: For every optimal path, record which tiles are used.

3. **Aggregate results**: Combine all tiles from all optimal paths, ensuring each tile is only counted once.

Here's a refined approach:

1. **Modify A* to track states**: Besides tracking the cost, track the sequence of tiles used for each optimal path.
2. **Explore all equivalent paths**: When a path ends with an optimal score, store the path information.
3. **Aggregate unique tiles**: Once all optimal paths are known, aggregate the set of unique tiles used in these paths.

Here's a Python implementation:

```python
import heapq

def parse_maze(maze):
    start = None
    end = None
    grid = []
    for y, line in enumerate(maze.strip().split('\n')):
        row = list(line)
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)
        grid.append(row)
    return grid, start, end

def heuristic(pos, end):
    # Use the Manhattan distance
    return abs(pos[0] - end[0]) + abs(pos[1] - end[1])

def get_neighbors(pos, direction, grid):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # East, North, West, South
    x, y = pos
    neighbors = []

    # Move forward
    dx, dy = directions[direction]
    nx, ny = x + dx, y + dy
    if 0 <= nx < len(grid[0]) and 0 <= ny < len(grid) and grid[ny][nx] != '#':
        neighbors.append(((nx, ny), direction, 1))  # Forward move cost

    # Rotate clockwise
    neighbors.append(((x, y), (direction + 1) % 4, 1000))

    # Rotate counter-clockwise
    neighbors.append(((x, y), (direction - 1) % 4, 1000))

    return neighbors

def find_best_paths_tiles(maze):
    grid, start, end = parse_maze(maze)
    initial_direction = 0  # Start facing East
    start_state = (start, initial_direction, 0, {(start)})

    # Priority queue for A*
    priority_queue = []
    heapq.heappush(priority_queue, (0 + heuristic(start, end), 0, start_state))

    best_score = float('inf')
    best_paths = []

    while priority_queue:
        _, current_cost, (current_pos, current_direction, _, path_tiles) = heapq.heappop(priority_queue)

        # If we have reached the end and this path is optimal
        if current_pos == end:
            if current_cost < best_score:
                best_score = current_cost
                best_paths = [path_tiles]
            elif current_cost == best_score:
                best_paths.append(path_tiles)
            continue

        # Explore neighbors
        for neighbor_pos, new_direction, move_cost in get_neighbors(current_pos, current_direction, grid):
            new_cost = current_cost + move_cost
            new_path_tiles = path_tiles | {neighbor_pos}

            if new_cost <= best_score:
                new_state = (neighbor_pos, new_direction, new_cost, new_path_tiles)
                heapq.heappush(priority_queue, (new_cost + heuristic(neighbor_pos, end), new_cost, new_state))

    # Aggregate all unique tiles part of any best path
    unique_tiles = set()
    for path in best_paths:
        unique_tiles.update(path)

    return len(unique_tiles), unique_tiles

# Example usage
maze_input1 = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""

num_tiles, tile_set = find_best_paths_tiles(maze_input1)
print(f"Number of tiles part of any best path: {num_tiles}")

maze_input2 = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
"""

num_tiles, tile_set = find_best_paths_tiles(maze_input2)
print(f"Number of tiles part of any best path: {num_tiles}")
```

This code will identify the number of unique tiles that are part of at least one of the best scoring paths. Using sets to track tiles for each path ensures that only unique tiles are counted.

## User

Great this approach works, except that it is way to slow, is there a way of creating a graph of the routing problem and pruning all paths which have no turns available due to higher costs to get to this earlier node and breaking based on that, after which you can count all tiles that have the same cost to reach a node x,y in the system

## Assistant

Indeed, we can optimize the solution by leveraging a graph-like exploration while pruning paths that exceed the optimal score for any given node. This can be achieved using a combination of breadth-first search (BFS) or a similar approach with careful tracking of the best cost at each node.

Here's the plan:

1. **Use a Priority Queue:** This approach will still benefit from a priority queue (like in the A* algorithm) to always explore the lowest cost nodes next. However, it will store all nodes even if they have the same cost, not discarding them immediately but instead processing them intelligently.

2. **Track Best Costs:** We'll maintain a dictionary to keep the lowest cost to reach each position and direction. If a new path reaches the same position and direction with a lower or equal cost, it can be deemed a potential optimal path.

3. **Prune Strategies:** During exploration, if a move adds to the path cost that exceeds the best-known path cost, that move is abandoned early.

4. **Aggregate All Optimal Paths:** After completing the exploration, we'll aggregate all tiles from any optimal (equal lowest cost) path to ensure all potential positions are counted.

5. **Avoid Duplicate Expansions:** Ensure that for each node, the algorithm attempts to expand a path to the node only if the current path cost is less than or equal to previously found paths to the same node.

Here's the updated Python implementation:

```python
import heapq
from collections import defaultdict

def parse_maze(maze):
    start = None
    end = None
    grid = []
    for y, line in enumerate(maze.strip().split('\n')):
        row = list(line)
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)
        grid.append(row)
    return grid, start, end

def heuristic(pos, end):
    return abs(pos[0] - end[0]) + abs(pos[1] - end[1])

def get_neighbors(pos, direction, grid):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # East, North, West, South
    x, y = pos
    neighbors = []

    dx, dy = directions[direction]
    nx, ny = x + dx, y + dy
    if 0 <= nx < len(grid[0]) and 0 <= ny < len(grid) and grid[ny][nx] != '#':
        neighbors.append(((nx, ny), direction, 1))

    neighbors.append(((x, y), (direction + 1) % 4, 1000))
    neighbors.append(((x, y), (direction - 1) % 4, 1000))

    return neighbors

def find_best_paths_tiles(maze):
    grid, start, end = parse_maze(maze)
    initial_state = (start, 0)  # Start at initial direction East
    priority_queue = []
    heapq.heappush(priority_queue, (0, 0, initial_state, {start}))

    best_costs = defaultdict(lambda: float('inf'))
    results = set()

    while priority_queue:
        current_score, current_cost, (current_pos, current_direction), path_tiles = heapq.heappop(priority_queue)

        if current_pos == end:
            results.update(path_tiles)
            continue

        if best_costs[(current_pos, current_direction)] < current_cost:
            continue

        best_costs[(current_pos, current_direction)] = current_cost

        for neighbor_pos, new_direction, move_cost in get_neighbors(current_pos, current_direction, grid):
            new_cost = current_cost + move_cost
            new_path_tiles = path_tiles | {neighbor_pos}

            if new_cost < best_costs[(neighbor_pos, new_direction)]:
                best_costs[(neighbor_pos, new_direction)] = new_cost
                new_state = (neighbor_pos, new_direction)
                heapq.heappush(priority_queue, (new_cost + heuristic(neighbor_pos, end), new_cost, new_state, new_path_tiles))

    return len(results), results

# Example usage
maze_input1 = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""

num_tiles1, tile_set1 = find_best_paths_tiles(maze_input1)
print(f"Number of tiles part of any best path for maze 1: {num_tiles1}")

maze_input2 = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
"""

num_tiles2, tile_set2 = find_best_paths_tiles(maze_input2)
print(f"Number of tiles part of any best path for maze 2: {num_tiles2}")
```

This approach uses a priority queue to ensure efficient exploration of paths with the lowest current cost, while strategically pruning suboptimal paths. The tiles that belong to any of the optimal paths are calculated by tracking all expanding paths. Performance should greatly improve for larger mazes.

## User

Sadly these paths aren't correct some are more expensive whilst earlier changes which have the same costs are being skipped

## Assistant

To achieve an efficient solution that accurately finds the tiles part of all optimal paths, we need to improve the handling of the paths by ensuring that all states are properly explored even if they have the same cost as previously found states. The goal is to employ an algorithm that does not prematurely prune paths and tracks visited states but allows revisitation of nodes when new, equally optimal paths are found.

Here's an improved approach that addresses these issues:

1. **Track Best Scores by Position and Direction:** Ensure that for each node, all paths that reach it with the minimal cost are explored and recorded.

2. **Avoid Premature Pruning:** Avoid skipping states with equal costs to ensure all potential optimal paths are considered.

3. **Aggregate Unique Tiles:** Ensure that for every unique cost-efficient traversal, the tiles visited are aggregated into a single set representing the path.

Here is the code implementation:

```python
import heapq
from collections import defaultdict

def parse_maze(maze):
    start = None
    end = None
    grid = []
    for y, line in enumerate(maze.strip().split('\n')):
        row = list(line)
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)
        grid.append(row)
    return grid, start, end

def heuristic(pos, end):
    return abs(pos[0] - end[0]) + abs(pos[1] - end[1])

def get_neighbors(pos, direction, grid):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # East, North, West, South
    x, y = pos
    neighbors = []

    dx, dy = directions[direction]
    nx, ny = x + dx, y + dy
    if 0 <= nx < len(grid[0]) and 0 <= ny < len(grid) and grid[ny][nx] != '#':
        neighbors.append(((nx, ny), direction, 1))

    neighbors.append(((x, y), (direction + 1) % 4, 1000))
    neighbors.append(((x, y), (direction - 1) % 4, 1000))

    return neighbors

def find_best_paths_tiles(maze):
    grid, start, end = parse_maze(maze)
    initial_state = (start, 0)  # Start at initial direction East
    priority_queue = []
    heapq.heappush(priority_queue, (0, 0, initial_state, {start}))

    # Track best known costs for state positions and directions
    best_costs = defaultdict(lambda: float('inf'))
    path_involvement = defaultdict(set)

    while priority_queue:
        _, current_cost, (current_pos, current_direction), path_tiles = heapq.heappop(priority_queue)

        if current_pos == end:
            path_involvement[current_cost].update(path_tiles)
            continue

        # Explore neighbors
        for neighbor_pos, new_direction, move_cost in get_neighbors(current_pos, current_direction, grid):
            new_cost = current_cost + move_cost
            new_path_tiles = path_tiles | {neighbor_pos}

            if new_cost <= best_costs[(neighbor_pos, new_direction)]:
                best_costs[(neighbor_pos, new_direction)] = new_cost
                new_state = (neighbor_pos, new_direction)
                heapq.heappush(priority_queue, (new_cost + heuristic(neighbor_pos, end), new_cost, new_state, new_path_tiles))
                path_involvement[new_cost] |= new_path_tiles

    # Get tiles from the minimum cost to reach the end
    min_cost = min(path_involvement.keys())
    tiles_on_best_paths = path_involvement[min_cost]

    return len(tiles_on_best_paths), tiles_on_best_paths

# Example usage
maze_input1 = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""

num_tiles1, tile_set1 = find_best_paths_tiles(maze_input1)
print(f"Number of tiles part of any best path for maze 1: {num_tiles1}")

maze_input2 = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
"""

num_tiles2, tile_set2 = find_best_paths_tiles(maze_input2)
print(f"Number of tiles part of any best path for maze 2: {num_tiles2}")
```

This solution revises the previous approach by tracking each node's cost and direction, propagating only paths that maintain or improve the best-known costs. By using and updating the `path_involvement` dictionary, the function tracks which tiles are part of optimal paths, ensuring all minimal cost paths are properly reflected. This improves correctness while also being efficient in terms of avoiding unnecessary calculations.

## User

Okay this doesn't work; new approach.
1. Calculated the lowest costs for all squares within the maze
2. Find all routes from start to finish in which are valid for these lowest scores (multiple routes have equal scores)
3. Give back the set of all tiles hit by these routes
4. Give me the number of tiles

## Assistant

Let's approach the problem methodically following your outlined steps. We need to:

1. **Calculate the Lowest Costs for All Squares:** Use a modified Dijkstra or A* to find the shortest path score for reaching each tile from the start with the ability to move in all directions.

2. **Trace Back All Valid Routes with These Costs:** From the endpoint, trace back all paths that conform to the computed optimal scores at each tile.

3. **Identify All Unique Tiles in These Routes:** Aggregate all tiles involved in any of the optimal routes.

4. **Count These Unique Tiles:** This will give us the number of tiles that are touched by at least one of these optimal paths.

Let's dive into the implementation:

```python
import heapq
from collections import defaultdict, deque

def parse_maze(maze):
    start = None
    end = None
    grid = []
    for y, line in enumerate(maze.strip().split('\n')):
        row = list(line)
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)
        grid.append(row)
    return grid, start, end

def dijkstra(grid, start):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # East, North, West, South
    dir_cost = [1, 1, 1, 1]  # all forward moves cost 1
    rotate_cost = 1000

    # Use priority queue for Dijkstra
    pq = []
    heapq.heappush(pq, (0, start, 0))  # (cost, position, direction)
    
    # Store the lowest cost to each cell
    cost_map = defaultdict(lambda: float('inf'))
    cost_map[(start, 0)] = 0
    
    while pq:
        current_cost, (x, y), direction = heapq.heappop(pq)
        
        if cost_map[((x, y), direction)] < current_cost:
            continue
        
        # Move forward in the current direction
        dx, dy = directions[direction]
        nx, ny = x + dx, y + dy

        if 0 <= nx < len(grid[0]) and 0 <= ny < len(grid) and grid[ny][nx] != '#':
            new_cost = current_cost + dir_cost[direction]
            if new_cost < cost_map[((nx, ny), direction)]:
                cost_map[((nx, ny), direction)] = new_cost
                heapq.heappush(pq, (new_cost, (nx, ny), direction))
        
        # Rotate clockwise
        new_direction = (direction + 1) % 4
        if current_cost + rotate_cost < cost_map[((x, y), new_direction)]:
            cost_map[((x, y), new_direction)] = current_cost + rotate_cost
            heapq.heappush(pq, (current_cost + rotate_cost, (x, y), new_direction))
        
        # Rotate counter-clockwise
        new_direction = (direction - 1) % 4
        if current_cost + rotate_cost < cost_map[((x, y), new_direction)]:
            cost_map[((x, y), new_direction)] = current_cost + rotate_cost
            heapq.heappush(pq, (current_cost + rotate_cost, (x, y), new_direction))
    
    return cost_map

def find_best_paths_tiles(maze_input):
    grid, start, end = parse_maze(maze_input)
    cost_map = dijkstra(grid, start)
    
    # Determine the minimal cost to reach the end
    min_cost = min(cost_map[(end, d)] for d in range(4))
    
    # Find all paths with this minimal cost
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]
    valid_tiles = set()

    # BFS/DFS to find all tiles in the shortest path traces
    queue = deque([(end, d, set()) for d in range(4) if cost_map[(end, d)] == min_cost])

    while queue:
        (x, y), direction, path = queue.popleft()
        
        if (x, y) == start:
            valid_tiles.update(path)
            continue

        # Move forward in reverse (check valid movements back in the current direction state)
        for d, (dx, dy) in enumerate(directions):
            nx, ny = x - dx, y - dy
            if (nx, ny) in path or not (0 <= nx < len(grid[0]) and 0 <= ny < len(grid)):
                continue
            needed_cost = cost_map[((x, y), direction)] - 1
            if cost_map.get(((nx, ny), direction), float('inf')) == needed_cost:
                queue.append(((nx, ny), direction, path | {(x, y)}))

        # Rotate back (reverse logic on how we could have arrived here)
        left_needed_cost = cost_map[((x, y), direction)] - 1000
        right_needed_cost = cost_map[((x, y), direction)] - 1000

        left_dir = (direction + 1) % 4
        right_dir = (direction - 1) % 4

        if cost_map.get(((x, y), left_dir), float('inf')) == left_needed_cost:
            queue.append(((x, y), left_dir, path | {(x, y)}))
        
        if cost_map.get(((x, y), right_dir), float('inf')) == right_needed_cost:
            queue.append(((x, y), right_dir, path | {(x, y)}))
    
    return len(valid_tiles) + 2  # Include start and end tiles

# Testing the solution with the examples provided
maze_input1 = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""

maze_input2 = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
"""

print(f"Number of tiles part of any best path for maze 1: {find_best_paths_tiles(maze_input1)}")
print(f"Number of tiles part of any best path for maze 2: {find_best_paths_tiles(maze_input2)}")
```

### Explanation:
1. **First Pass (Dijkstra/Modified Dijkstra):** Computes the minimum cost to reach every tile with all possible directions from the start.
2. **Backtracking Phase:** Uses a queue to backtrack from the end tile to the start tile, determining all optimal paths by following the minimal cost map.
3. **Tile Aggregation:** Collects all unique tiles included in any optimal path.

This approach ensures a comprehensive evaluation by respecting and fully exploring all minimally-costed path routes through backtracking based on computed costs.

## User

Top this works however it is extremly slow can't we simplify it by just checking backwards per square from the end if the cost is lower then the current cost till we hit the start? And just adding all passed squares in a set called visited routes

## Assistant

Certainly! We can streamline the approach by employing a backward tracing technique starting from the end position. This involves reversing the logic used in the initial cost calculation to only explore paths that maintain the optimal cost. Here's how we can simplify it:

1. **Forward Path Calculation:** We'll use a modified Dijkstra's algorithm to calculate the optimal cost for each tile starting from the 'S' tile with all possible directions.

2. **Backward Exploratory Search:** Starting from the 'E' tile, we'll trace back using the calculated cost map, ensuring that we only traverse edges that maintain or reduce the optimal cost. In doing so, we'll collect tiles that form any optimal path in a visited set.

Here's a streamlined implementation:

```python
import heapq
from collections import defaultdict, deque

def parse_maze(maze):
    start = None
    end = None
    grid = []
    for y, line in enumerate(maze.strip().split('\n')):
        row = list(line)
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)
        grid.append(row)
    return grid, start, end

def calculate_costs(grid, start):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # East, North, West, South
    rotate_cost = 1000

    # Priority queue for Dijkstra-like exploration
    pq = []
    # (cost, position, direction)
    heapq.heappush(pq, (0, start, 0))
    
    # Store the cost to each cell with direction
    cost_map = defaultdict(lambda: float('inf'))
    cost_map[(start, 0)] = 0

    while pq:
        current_cost, (x, y), direction = heapq.heappop(pq)
        
        if cost_map[((x, y), direction)] < current_cost:
            continue

        # Move forward in current direction
        dx, dy = directions[direction]
        nx, ny = x + dx, y + dy
        if 0 <= nx < len(grid[0]) and 0 <= ny < len(grid) and grid[ny][nx] != '#':
            new_cost = current_cost + 1
            if new_cost < cost_map[((nx, ny), direction)]:
                cost_map[((nx, ny), direction)] = new_cost
                heapq.heappush(pq, (new_cost, (nx, ny), direction))
        
        # Rotate clockwise
        new_direction = (direction + 1) % 4
        if current_cost + rotate_cost < cost_map[((x, y), new_direction)]:
            cost_map[((x, y), new_direction)] = current_cost + rotate_cost
            heapq.heappush(pq, (current_cost + rotate_cost, (x, y), new_direction))
        
        # Rotate counter-clockwise
        new_direction = (direction - 1) % 4
        if current_cost + rotate_cost < cost_map[((x, y), new_direction)]:
            cost_map[((x, y), new_direction)] = current_cost + rotate_cost
            heapq.heappush(pq, (current_cost + rotate_cost, (x, y), new_direction))

    return cost_map

def trace_optimal_paths(grid, end, cost_map):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # East, North, West, South
    rotate_cost = 1000
    valid_tiles = set()

    # Start from each direction at the end that reaches end with minimal cost
    min_cost = min(cost_map[(end, d)] for d in range(4))
    stack = [(end, d) for d in range(4) if cost_map[(end, d)] == min_cost]

    while stack:
        (x, y), direction = stack.pop()
        valid_tiles.add((x, y))

        # Backtrack to find all predecessors
        # Check if moving forward is possible
        dx, dy = directions[direction]
        px, py = x - dx, y - dy
        
        # If moving backward matches optimal cost, append
        if (0 <= px < len(grid[0]) and 0 <= py < len(grid)
            and cost_map[(px, py, direction)] + 1 == cost_map[(x, y, direction)]):
            stack.append(((px, py), direction))
        
        # Check rotation from previous direction
        for rot, new_direction in [(1, (direction - 1) % 4), (-1, (direction + 1) % 4)]:
            if cost_map[(x, y, direction)] == cost_map[(x, y, new_direction)] + rotate_cost:
                stack.append(((x, y), new_direction))
                
    return valid_tiles

def find_best_paths_tiles(maze_input):
    grid, start, end = parse_maze(maze_input)
    cost_map = calculate_costs(grid, start)
    valid_tiles = trace_optimal_paths(grid, end, cost_map)
    return len(valid_tiles)

# Test examples
maze_input1 = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""

maze_input2 = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
"""

print(f"Number of tiles part of any best path for maze 1: {find_best_paths_tiles(maze_input1)}")
print(f"Number of tiles part of any best path for maze 2: {find_best_paths_tiles(maze_input2)}")
```

### Key Improvements:
- **Efficiency in Calculation:** Backward tracing from the endpoint considers only viable paths maintaining the optimal calculated cost, which is more efficient.
- **Unified Tile Collection:** Collects all unique tiles visited by such optimal paths.
- **Reduced Operations:** Reduces excess operations by only expanding necessary nodes on valid trajectories.

These refinements should significantly enhance the speed, maintaining correctness while ensuring all valid paths are fully considered.

## User

Can you add a skip clause that if the item of the stack has been visited before it is just skipped

## Assistant

Certainly! We can enhance the backward path tracing by ensuring that each state (position and direction) is only processed once. This not only avoids unnecessary recomputation but also significantly enhances performance by skipping redundant evaluations. Here's how we can incorporate this improvement:

1. **Visited Set:** We'll introduce a set that keeps track of visited states during backward tracing. Before processing a state, we'll check if it has been visited; if so, we'll skip it.

2. **Efficient Pruning:** This will prevent the algorithm from revisiting the same node-direction combination, thus improving time efficiency.

Here's the updated implementation with this optimization:

```python
import heapq
from collections import defaultdict

def parse_maze(maze):
    start = None
    end = None
    grid = []
    for y, line in enumerate(maze.strip().split('\n')):
        row = list(line)
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)
        grid.append(row)
    return grid, start, end

def calculate_costs(grid, start):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # East, North, West, South
    rotate_cost = 1000

    pq = []
    heapq.heappush(pq, (0, start, 0))
    
    cost_map = defaultdict(lambda: float('inf'))
    cost_map[(start, 0)] = 0

    while pq:
        current_cost, (x, y), direction = heapq.heappop(pq)
        
        if cost_map[((x, y), direction)] < current_cost:
            continue

        dx, dy = directions[direction]
        nx, ny = x + dx, y + dy
        if 0 <= nx < len(grid[0]) and 0 <= ny < len(grid) and grid[ny][nx] != '#':
            new_cost = current_cost + 1
            if new_cost < cost_map[((nx, ny), direction)]:
                cost_map[((nx, ny), direction)] = new_cost
                heapq.heappush(pq, (new_cost, (nx, ny), direction))
        
        new_direction = (direction + 1) % 4
        if current_cost + rotate_cost < cost_map[((x, y), new_direction)]:
            cost_map[((x, y), new_direction)] = current_cost + rotate_cost
            heapq.heappush(pq, (current_cost + rotate_cost, (x, y), new_direction))
        
        new_direction = (direction - 1) % 4
        if current_cost + rotate_cost < cost_map[((x, y), new_direction)]:
            cost_map[((x, y), new_direction)] = current_cost + rotate_cost
            heapq.heappush(pq, (current_cost + rotate_cost, (x, y), new_direction))

    return cost_map

def trace_optimal_paths(grid, end, cost_map):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]
    rotate_cost = 1000
    valid_tiles = set()
    visited = set()

    min_cost = min(cost_map[(end, d)] for d in range(4))
    stack = [(end, d) for d in range(4) if cost_map[(end, d)] == min_cost]

    while stack:
        (x, y), direction = stack.pop()
        
        if ((x, y), direction) in visited:
            continue
        
        visited.add(((x, y), direction))
        valid_tiles.add((x, y))

        dx, dy = directions[direction]
        px, py = x - dx, y - dy
        
        if (0 <= px < len(grid[0]) and 0 <= py < len(grid)
            and cost_map.get(((px, py), direction), float('inf')) + 1 == cost_map[(x, y, direction)]):
            stack.append(((px, py), direction))
        
        for rot, new_direction in [(1, (direction - 1) % 4), (-1, (direction + 1) % 4)]:
            if cost_map.get((x, y, direction), float('inf')) == cost_map.get((x, y, new_direction)) + rotate_cost:
                stack.append(((x, y), new_direction))
                
    return valid_tiles

def find_best_paths_tiles(maze_input):
    grid, start, end = parse_maze(maze_input)
    cost_map = calculate_costs(grid, start)
    valid_tiles = trace_optimal_paths(grid, end, cost_map)
    return len(valid_tiles)

# Test cases
maze_input1 = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""

maze_input2 = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
"""

print(f"Number of tiles part of any best path for maze 1: {find_best_paths_tiles(maze_input1)}")
print(f"Number of tiles part of any best path for maze 2: {find_best_paths_tiles(maze_input2)}")
```

### Explanation:
- **Visited Set:** The `visited` set keeps track of states that have been processed. 
- **Skip Clause:** Before processing a state, check if it's in the `visited` set to determine whether to skip it, ensuring each state is only processed once.
- **Efficient Exploration:** This constraint significantly optimizes the backtracking process, preventing redundant computations and ensuring a more efficient execution. 

This refined implementation should speed up the process by ensuring you don't revisit states unnecessarily.

## User

You are indexing the cost_map wrong, you now have a tuple (x,y,dir) however it should be ((x,y),dir)

## Assistant

Thank you for pointing that out. Indeed, the indexing of the `cost_map` should correctly use `((x, y), direction)` instead of `(x, y, direction)`. Here is the corrected code with appropriate indexing:

```python
import heapq
from collections import defaultdict

def parse_maze(maze):
    start = None
    end = None
    grid = []
    for y, line in enumerate(maze.strip().split('\n')):
        row = list(line)
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)
        grid.append(row)
    return grid, start, end

def calculate_costs(grid, start):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # East, North, West, South
    rotate_cost = 1000

    pq = []
    heapq.heappush(pq, (0, start, 0))
    
    cost_map = defaultdict(lambda: float('inf'))
    cost_map[(start, 0)] = 0

    while pq:
        current_cost, (x, y), direction = heapq.heappop(pq)
        
        if cost_map[((x, y), direction)] < current_cost:
            continue

        dx, dy = directions[direction]
        nx, ny = x + dx, y + dy
        if 0 <= nx < len(grid[0]) and 0 <= ny < len(grid) and grid[ny][nx] != '#':
            new_cost = current_cost + 1
            if new_cost < cost_map[((nx, ny), direction)]:
                cost_map[((nx, ny), direction)] = new_cost
                heapq.heappush(pq, (new_cost, (nx, ny), direction))
        
        new_direction = (direction + 1) % 4
        if current_cost + rotate_cost < cost_map[((x, y), new_direction)]:
            cost_map[((x, y), new_direction)] = current_cost + rotate_cost
            heapq.heappush(pq, (current_cost + rotate_cost, (x, y), new_direction))
        
        new_direction = (direction - 1) % 4
        if current_cost + rotate_cost < cost_map[((x, y), new_direction)]:
            cost_map[((x, y), new_direction)] = current_cost + rotate_cost
            heapq.heappush(pq, (current_cost + rotate_cost, (x, y), new_direction))

    return cost_map

def trace_optimal_paths(grid, end, cost_map):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]
    rotate_cost = 1000
    valid_tiles = set()
    visited = set()

    min_cost = min(cost_map[(end, d)] for d in range(4))
    stack = [(end, d) for d in range(4) if cost_map[(end, d)] == min_cost]

    while stack:
        (x, y), direction = stack.pop()
        
        if ((x, y), direction) in visited:
            continue
        
        visited.add(((x, y), direction))
        valid_tiles.add((x, y))

        dx, dy = directions[direction]
        px, py = x - dx, y - dy
        
        if (0 <= px < len(grid[0]) and 0 <= py < len(grid)
            and cost_map.get(((px, py), direction), float('inf')) + 1 == cost_map[((x, y), direction)]):
            stack.append(((px, py), direction))
        
        for rot, new_direction in [(1, (direction - 1) % 4), (-1, (direction + 1) % 4)]:
            if cost_map.get(((x, y), new_direction), float('inf')) + rotate_cost == cost_map[((x, y), direction)]:
                stack.append(((x, y), new_direction))
                
    return valid_tiles

def find_best_paths_tiles(maze_input):
    grid, start, end = parse_maze(maze_input)
    cost_map = calculate_costs(grid, start)
    valid_tiles = trace_optimal_paths(grid, end, cost_map)
    return len(valid_tiles)

# Test cases
maze_input1 = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""

maze_input2 = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
"""

print(f"Number of tiles part of any best path for maze 1: {find_best_paths_tiles(maze_input1)}")
print(f"Number of tiles part of any best path for maze 2: {find_best_paths_tiles(maze_input2)}")
```

### Explanation:
- **Corrected Indexing:** The `cost_map` is now indexed correctly using the tuple structure `((x, y), direction)`.
- **General Structure:** The approach is similar, but ensures accurate referencing and accumulation of state data during computation.
- **Efficiency:** The corrected indexing helps ensure that we're consistently interpreting and evaluating the correct states, correcting the efficiency and accuracy of our pathfinding and backtracking approaches.

## User

Great everything works

